# Social Views

These questions asked respondents their opinion about various social issues and what role they thought the government should take in addressing those issues.  They were also asked for their opinions about the trustworthiness of people in general and politicians in particular.

In [ ]:
# Load pre-processed data
!pip install joblib
import joblib

import plotly.express as px

import pandas as pd
import numpy as np
from math import floor, ceil, log

data = joblib.load('GroupedAndUngroupedData.pkl')

Function used to easily generate histograms

In [ ]:
def ShowHistogram (data, col, sortorder='ascending', sorttype='category', **kwargs):
    """
    Function to display histograms and percentage breakdowns for a specified column in the dataframe.
    @ params:
        data                        -  dataframe 
        col                         -  column name within dataframe
        sortorder                   -  ascending or descending (default is ascending)
        sorttype                    -  sort either by category name ('category') or totals ('total'). Default is category.
        **kwargs                    -  optional additional arguments to pass into the histogram call. for example, adding
                                       facet_col = 'gender_Groups' will additionally facet the histogam by gender 
    """
    
    tickmode='array'
    ascending = sortorder == 'ascending'
    
    # If we're faceting by something, we want to also sort by the facet column so the results are displayed consistently
    # (Special case for age so that the age groups are sorted in numerical rather than alphabetical order)
    if 'facet_col' in kwargs.keys():
        if kwargs['facet_col'] == 'age_Groups':
            data = data.sort_values([col,'age'], ascending=ascending)
        else:
            data = data.sort_values([col,kwargs['facet_col']], ascending=ascending)
    else:
        data = data.sort_values([col], ascending=ascending)
    
    tickvals=data[col]
    
    # Our preprocessed data has columns representing the responses and then our grouped / translated versions of those
    # responses as ""{columnName}_Groups".  However, not every column was Grouped (age, for example).
    # If the column we're generating the histogram for does have a _Groups equivalent, we'll display those values
    # in the X axis.  Otherwise, we'll just use the data as-is.
    if f'{col}_Groups' in data.columns:
        ticktext=data[f'{col}_Groups']
        vals = sorted(set(data[f'{col}_Groups']))
    else:
        ticktext=data[col]
        vals = sorted(set(data[f'{col}']))
    
    # To help ensure the colors in the histograms are consistent, this part maps each of the unique values being 
    # graphed to a color in the colors.qualitative.G10 pallette (in PlotLy).  This only works if there are 
    # 10 or fewer unique values (since the qualitative pallettes only have 10 values).  So if there are more, we just skip
    # this step.
    colorDict = {}
    
    if len (vals) <= 10:
        for val in vals:
            colorDict[val] = px.colors.qualitative.G10[len(colorDict)]
    if colorDict:
        p=px.histogram(data,x=col, color=ticktext, histnorm='density',color_discrete_map = colorDict, **kwargs)
    else:
        p=px.histogram(data,x=col, color=ticktext, histnorm='density',**kwargs)
        
    # Remove the "colname=" junk from the legends
    p=p.for_each_trace(lambda t: t.update(name=t.name.split('=')[1]))
    
    # If the responses were grouped 1:1, then we can use the raw responses on the x-axis but display their translated
    # names
    if len(set(tickvals)) == len(set(ticktext)):
        p.update_layout(xaxis=dict(tickmode=tickmode, tickvals=tickvals, ticktext=ticktext))
    
    # Remove the "colname=" junk from the facet labels
    if 'facet_col' in kwargs.keys():
        p = p.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    
    # Sort the x axis in the specified order
    categoryorder = f'{sorttype} {sortorder}'
    p=p.update_xaxes(type='category', categoryorder=categoryorder)
    p.show()
    
    if f'{col}_Groups' in data.columns:
        print ('Grouped Columns Distribution:\n')
        print (data[f'{col}_Groups'].value_counts(normalize=True)*100)
        print ('\n')
    print ('Ungrouped Columns Distribution:\n')
    print (data[f'{col}'].value_counts(normalize=True)*100)

# Trust In Other People

This series of questions asked respondents to provide their opinions on the trustworthiness of other people.  In general, the most common response to these questions tended to be "neither agree nor disagree", so these questions may not offer much insight.

This question asked them how strongly they agreed or disagreed with the statement: "Generally speaking, most people can be trusted"

The responses are close to a normal distribution, though skewed right (towards agreement) slightly.  The most common answer was "neither agree nor disagree", so this question may not provide much insight.  However, when it came to politicians, most respondents seemed to indicate a general lack of confidence or trust.

In [ ]:
col = 'peopleTrusted'

ShowHistogram (data, col)

Respondents in this question were asked to indicate how much they agree/disagree with the statement:
"Most people try to take advantage of you if they get the chance"

Responses here tend to skew right (positive) overall, though neutral is still a very common response.

In [ ]:
ShowHistogram (data,'peopleTakeAdvantage')

Respondents in this question were asked to indicate how much they agree/disagree with the statement:
"Most people try to be fair"

Similar to the previous question, responses here tend to skew right (positive) overall, but here neutrality is the most common response.

In [ ]:
ShowHistogram (data,'peopleFair')

The next set of items asked about respondents' feelings about government officials and elected representatives.

This question asked:

How often can you trust the people who run our government to do what is right?

We see here that responses tended to lean towards the low end - 71% responded either "hardly ever" or "some of the time".  Very few responded "most of the time" or "always".

In [ ]:
ShowHistogram (data, 'politiciansDoRight')

This question asked:

How often do you think that government officials and elected representatives make decisions in the best interests of the country as a whole?

Similar to the last question, most expressed low confidence in this statement.

In [ ]:
ShowHistogram(data, 'politiciansCountryBestInterests')

This question asked:

When our government leaders make public statements on television and in newspapers, how often do you think they are telling the truth?

Again, generally low confidence, but slightly more responses of "most of the time" than the prior two questions.

In [ ]:
ShowHistogram(data, 'politiciansTellTruth')

# Trust In News Media

This series of questions asked respondents their opinions about the trustworthiness of the new media.  In general, respondents felt that the news media was not trustworthy (or at best did not have a strong opinion one way or the other).

Respondents in this question were asked to indicate how much they agree/disagree with the statement:

The news media generally report on political issues and affairs in an unbiased manner."

This data skews left (towards disagreement), though neutrality is still the most common response.  
Interestingly, females tended to disgaree with this statement more often than males.

In [ ]:
ShowHistogram (data,'newsUnbiased', facet_col='gender_Groups')

Respondents in this question were asked to indicate how much they agree/disagree with the statement:

"In general, the news media can be trusted to provide accurate information about politics and social issues."

This data also skews towards disagreement, though again, neutral is the most common response.
Again, females tended towards disagreement more often than men.

In [ ]:
ShowHistogram (data,'newsAccurate', facet_col='gender_Groups')

Respondents in this question were asked to indicate how much they agree/disagree with the statement:

"Most members of the news media are trustworthy"

This distribution is almost normal, though it skews towards the left (disagreement) slightly.

In [ ]:
ShowHistogram (data,'newsTrustworthy')

# Various Statements Regarding Social, Political, and Economic Issues

These questions asked respondents to give their opinions on the role of government in general, and also asked about certain social/political issues.  When it comes to the role of the government, responses to these questions indicate what might be called a "conservative" tendency in many of the respondents.  Unsurprisingly, the topic of abortion was especially polarizing. Many social issues, however, had a more progressive lean.

Respondents in this question were asked to indicate how much they agree/disagree with the statement:

"The government should help more vulnerable people, even if it means going deeper into debt"

The distribution of responses here is almost normal, though skews slightly towards to the left (towards disagreement).

In [ ]:
ShowHistogram (data,'govtHelpVulnerable')

Respondents in this question were asked to indicate how much they agree/disagree with the statement:
    
"Government regulation of business usually does more harm than good"

This data skews to the right (towards agreement)

In [ ]:
ShowHistogram (data,'govtBusinessRegulation')

Respondents in this question were asked to indicate how much they agree/disagree with the statement:
    
"Poor people have become too dependent on government assistance programs"

There is a very strong skew towards the right (agreement) on this question.

In [ ]:
ShowHistogram (data,'govtPoorDependence')

Respondents in this question were asked to indicate how much they agree/disagree with the statement:

"The growing number of newcomers from other countries threatens traditional American values"

Mostly the sentiment is neutral overall, but there is a slight trend towards disagreement.  However, when we facet by gender, we see that the second most common response for females is disagreement, whereas for males the second most common response is agreement.

Additionally, faceting by age groups, we see that the sentiment tends to shift from disagreement (i.e., a more progressive stance) towards agreement (i.e., a more conservative stance) as the respondents get older.  For those in their late 20s, agreement was the most common response.

In [ ]:
ShowHistogram (data,'immigrationThreatenTradValues', facet_col='gender_Groups')

In [ ]:
ShowHistogram (data,'immigrationThreatenTradValues', facet_col='age_Groups')

Respondents in this question were asked to indicate how much they agree/disagree with the statement:

"Gays and lesbians should be allowed to marry legally"

Here we see a strong trend towards agreement.

In [ ]:
ShowHistogram (data,'gayMarriage')

Respondents in this question were asked to indicate how much they agree/disagree with the statement:

"Abortion should be illegal in all or most cases"

Responses here seem split.  Approximately the same number disagreed (strongly or otherwise) as those who agreed (strongly or otherwise).  Strongly disagree is the most common answer, but only barely. Neutral and Strongly agree follow close behind. Faceting by age groups provides some more nuance.  We can see that, in fact, strongly disagree is the most common response in all age groups. However, those who did not strongly disagree tended to either be neutral or agree (though perhaps not strongly).  This indicates a somewhat conservative "lean" in the respondents' attitudes, but maybe not a strong one.

In [ ]:
ShowHistogram(data, 'abortionLegal')

In [ ]:
ShowHistogram(data, 'abortionLegal', facet_col='age_Groups')

Respondents in this question were asked to indicate how much they agree/disagree with the statement:

"Protests are a normal part of democracy"

There appears to be a strong trend towards agreement here -- 67% agreed to some degree.

In [ ]:
ShowHistogram(data, 'protestNormal')

Respondents in this question were asked to indicate how much they agree/disagree with the statement:

"Protests can be an effective way for ordinary citizens to voice their opinion on political issues"

This appears to be a similar distribution as the previous question -- mostly agreement.

In [ ]:
ShowHistogram (data,'protestEffective')

Respondents in this question were asked to indicate how much they agree/disagree with the statement:

"Protests are not representative of the political views of most ordinary people"

Respondents did not seem to have strong feelings about this question -- neutral (neither agree nor disagree) was the most common response.

In [ ]:
ShowHistogram (data,'protestNotRepresentative')

Respondents in this question were asked to indicate how much they agree/disagree with the statement:

"Protests should be illegal"

Most (~64%) disagreed with this statement.

In [ ]:
ShowHistogram (data,'protestIllegal')

# Activities related to community causes

These questions asked respondents if they did certain activites in the past year related to taking action to effect social change.  Specifically, did they take such action online, offline, both, or neither.  We were particularly interested to see if political / social activity was done online vs offline, so we categorized "both" as online.  Unfortunately, the overall trend here indicates that respondents generally did not take any action, either online or offline.

This question asked if respondents had done the following in the past year:

Raised money for a charity or ran/walked/biked for charity

No seems to be the most common answer.

In [ ]:
ShowHistogram (data, 'pyRaisedMoneyForCharity')

This question asked if respondents had done the following in the past year:

Made a donation to a political or social cause

Again, the most common answer was no.

In [ ]:
ShowHistogram(data, 'pyDonatedMoneyForCause')

This question asked if respondents had done the following in the past year:

Signed up to receive information about social or political causes.

Again, the most common answer is no.  However, here we see that more people were willing to perform this activity online than the prior activities -- ~30% indicated they signed up for information online.

In [ ]:
ShowHistogram(data, 'pySignedUpForCauseInfo')

This question asked if respondents had done the following in the past year:

Wore or displayed a sign or symbol related to a political or social cause

Here we see that very few people performed this activity, even online.

In [ ]:
ShowHistogram(data, 'pyDisplayedCauseSymbol')

This question asked if respondents had done the following in the past year:

Deliberately bought a product for political, ethical, or environmental reasons

Again, the most common answer is no.

In [ ]:
ShowHistogram(data, 'pyBoughtForCause')

This question asked if respondents had done the following in the past year:

Deliberately avoided buying a product (boycotted) for political, ethical, or environmental reasons

Again, the most common answer is no.  There is a slight uptick here in people who indicated they performed this activity online.

In [ ]:
ShowHistogram(data, 'pyBoycottedForCause')